# DAMAGE MULTI CLASSIFICATION

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle
import torch

In [24]:
data = pd.read_csv("C:/Users/Admin/Desktop/Coding/Tensorflow/csv_building_damage_assessment.csv",na_values = ["None"," ","nan","\n"])
data = data.drop(["building_id","district_id","vdcmun_id","ward_id"],axis = 1)
for column in data.columns:
    if data[column].count() <250000:
        data = data.drop(column,axis = 1)
        
data = data.dropna()

C:\Users\Admin\AppData\Local\Temp\ipykernel_1888\2382059404.py:1: DtypeWarning: Columns (5,17,29,34,35,36,37,38,39,40,41,42,43,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:/Users/Admin/Desktop/Coding/Tensorflow/csv_building_damage_assessment.csv",na_values = ["None"," ","nan","\n"])


## Assigning Features & Labels

In [25]:
features = pd.get_dummies(data) 
labels = data["damage_grade"]
classes = labels.unique()
labels = pd.get_dummies(labels)


In [26]:
features = np.array(features)
labels = np.array(labels)
features = ss().fit_transform(features)
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)
print(features.shape,labels.shape)

torch.Size([36017, 35]) torch.Size([36017, 5])


In [27]:
train_features = features[:30000]
train_labels = labels[:30000]
test_features = features[30000:]
test_labels = labels[30000:]

## Using DataLoader & TensorDataset

In [28]:
from torch.utils.data import DataLoader, TensorDataset

data = TensorDataset(train_features,train_labels)
BATCH_SIZE = 1000
dataloader = DataLoader(data,batch_size=BATCH_SIZE, shuffle=True)


### Using LeakyRelu & Softmax

In [29]:
model = torch.nn.Sequential(
    torch.nn.Linear(35,60),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(60,70),
    torch.nn.ReLU(),
    torch.nn.Linear(70,30),
    torch.nn.ReLU(),
    torch.nn.Linear(30,5), 
    torch.nn.Softmax(dim=-1) 
)

In [30]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
history = []

In [31]:
from torch import no_grad
epochs = 0
for epoch in range(epochs):
    for features,labels in dataloader:
        preds = model(train_features)
        loss = loss_fn(preds,train_labels)

        loss.backward()
        opt.step()
        opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            preds = model(test_features)
            for i in range(len(preds)):
                for j in range(len(preds[i])):
                        
                        if preds[i][j]==max(preds[i]):
                            if(test_labels[i][j]==1):
                                right+=1
                                print(test_labels[i],preds[i])
                
            Acc = round((right*100/len(test_labels)),5)
            history.append(Acc)
            print("Accuracy :",Acc)
        

*Saving Weight & Bias*

In [32]:
# torch.save(model.state_dict(),"damage.pth")


In [33]:
model.load_state_dict(torch.load("damage.pth"))

<All keys matched successfully>

**Result**

In [34]:
index = 999
pred = model(test_features[index])
pred_i = torch.where(pred == pred.max())
real_i = torch.where(test_labels[index] == 1)

print(f"Pred : {classes[pred_i]}, Actual : {classes[real_i]}")

Pred : Grade 1, Actual : Grade 1
